# X3FL dynamic relationships tool

In [1]:
# just run and ignore this block
import numpy as np
import pandas as pd
all_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran", "Yaki"]
# https://imgur.com/nqI0nbO, https://imgur.com/7BhbUMa, by blazenclaw, forum.egosoft.com
try:
    relationship_df = pd.read_csv("relationship.csv", index_col=0)
except:
    relationship_df = pd.read_csv("https://raw.githubusercontent.com/mkmark/X3FL-dynamic-relationships/main/relationship.csv", index_col=0)
relationship_df = pd.DataFrame(relationship_df, columns=all_races)
def get_X(races):
    R = np.array(relationship_df.loc[races, races])
    invR = np.linalg.inv(R) 
    N = np.ones(len(races))
    X = invR@N
    return X

def get_X_df(races):
    X = get_X(races)
    return pd.DataFrame(list(X)+[sum(X)]+[len(races)/sum(X)], index=races+['sum', 'efficiency']).transpose()

# https://stackoverflow.com/questions/26332412/python-recursive-function-to-display-all-subsets-of-given-set
def subs(l):
    if l == []:
        return [[]]

    x = subs(l[1:])

    return x + [[l[0]] + y for y in x]

def get_X_optimal_sub(races):
    possible_solution_racess = []
    possible_solution_Xs = []
    possible_solution_workloads = []
    for sub_races in subs(races):
        X = get_X(sub_races)
        R_all = np.array(relationship_df.loc[races, sub_races])
        N_all = R_all@X
        min_N_all = min(N_all)
        if min_N_all>0:
            if min_N_all<1:
                X = [x*1/min_N_all for x in X]
            possible_solution_racess += [sub_races]
            possible_solution_Xs += [X]
            possible_solution_workloads += [sum(X)]
    if len(possible_solution_workloads)>0:
        min_index = possible_solution_workloads.index(min(possible_solution_workloads))
        optiaml_solution_races = possible_solution_racess[min_index]
        optiaml_solution_X = possible_solution_Xs[min_index]
        if min(optiaml_solution_X)>0:
            return pd.DataFrame(list(optiaml_solution_X)+[sum(optiaml_solution_X)]+[len(optiaml_solution_races)/sum(optiaml_solution_X)], index=optiaml_solution_races+['sum', 'efficiency']).transpose()

In [2]:
# define target friend races as the following examples
#commonwealth_races = ["Teladi", "Argon", "Boron", "Split", "Paranid"]
#major_races = ["Goner", "Teladi", "NMMC", "Argon", "Boron", "Split", "Paranid", "Terran"]
#all_but_yaki_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran"]
#all_but_yaki_terran_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates"]
#all_but_yaki_terran_pirates_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid"]
#all_but_yaki_pirates_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Terran"]
#all_but_terran_pirates_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Yaki"]

In [3]:
races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid"]

In [4]:
# get result with get_X_optimal_sub(target_friend_races)
get_X_optimal_sub(races)

,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,sum,efficiency
0,7.780909,7.781132,13.390187,12.529926,10.355119,14.294295,17.964491,15.606104,18.489653,22.60303,21.615287,162.410133,0.06773


The number for each race represents how much effort you will have to put with that race to get 1 notoriety point for each race assuming you're to be friend with all these selected race.

The sum is the total workload of current tactic.

The efficiency is the sum of actual gained notoriety points (equal the number of races selected) divided by total workload.